In [32]:
#examing first 100 lines of file
i = 0
with open("movies.txt") as infile:
    for line in infile:
        i += 1
        if i == 100:
            break;
        print(line)
        

In [1]:
import networkx as nx

# handling large file, reading line by line, and pass line to functionByLine
def readfile(functionByLine):
    #read file (ignoring non-utf bytes)
    bufsize = 1024 * 1024 #1M
    with open("movies.txt", errors="ignore") as infile:
        while True:
            lines = infile.readlines(bufsize)
            if not lines:
                break
            for line in infile:
                functionByLine(line)

# read vertices procedure need global variable of "movieGraph", takes about 1 min for verteces
movieGraph = nx.Graph()
def readVertexByLine(line):
    if line.startswith("product/productId:"):
        lineSplited = line.split(":")
        movieID = lineSplited[1].strip()
        if movieID not in movieGraph:
            movieGraph.add_node(movieID)
# Adding vertices:
readfile(readVertexByLine)

# TODO add edges to graph

In [2]:
#Graph properties:
movieGraph.number_of_nodes()

253044

In [3]:
# TODO experiment 1: aggregation effect?
# edge adding processes:
# for each reviewers has n movies reviews 
#     => those n movies should connect to each other with (original weight+1)
# analysis this graph : clusters?  popularity?  diffusal?


# mapping reviewr to movies  #about 1 min
reviewer_Movies = {}
reviewer = None
movie = None
def readReviewerMoviesByLine(line):
    global reviewer
    global movie
    if line.startswith("product/productId:"):
        lineSplited = line.split(":")
        movie = lineSplited[1].strip()
    elif line.startswith("review/userId:"):
        lineSplited = line.split(":")
        reviewer = lineSplited[1].strip()
        if reviewer not in reviewer_Movies:
            reviewer_Movies[reviewer] = set()
        reviewer_Movies[reviewer].add(movie)
readfile(readReviewerMoviesByLine)


In [4]:
# number of reviewrs
len(reviewer_Movies)

889173

In [5]:
# looping of adding edges #experiment 1, about 2-3 mins

feature = "weight"
increment = 1
for reviewer in reviewer_Movies:
    for movies1 in reviewer:
        for movies2 in reviewer:
            if movies1 != movies2:
                if not movieGraph.has_edge(movies1, movies2):
                    movieGraph.add_edge(movies1, movies2)
                if feature not in movieGraph[movies1][movies2]:
                    movieGraph[movies1][movies2][feature] = increment
                else:
                    movieGraph[movies1][movies2][feature] += increment


In [6]:
#graph properties about edges

#only 728 edges?? this surprise me!
movieGraph.number_of_edges()


728

In [ ]:
#basic visualization of movie graph
nx.draw(movieGraph)
plt.show()

In [30]:
# TODO experiment 2: reviews features?
# edge adding processes:
# same as experiment 1 but each edge instead of adding weight, extracting the featrues lables of those reviews,
# adding the weight of corresonding features weight between those movies
